In [ ]:
secondary functions called by compareNet main function

the model's classifier's class

In [ ]:

class Classifier(nn.Module):
    def __init__(self, params, vocab_size, pte=None):
        super(Classifier, self).__init__()
        self.params = params
        self.vocab_size = vocab_size
        self.pte = False if pte is None else True

        self.text_encoder = TextEncoder(params)
        self.enti_encoder = EntityEncoder(params)
        # numOfEntity = 100000
        # self.enti_encoder = nn.Embedding(numOfEntity, params.hidden_dim)
        # nn.init.xavier_uniform_(self.enti_encoder.weight)
        self.topi_encoder = nn.Embedding(100, 100)
        self.topi_encoder.from_pretrained(torch.eye(100))
        self.match_encoder = MatchingTransform(params)
        # self.match_encoder = ConcatTransform(params)   # 参数试验用的
        self.word_embeddings = nn.Embedding(vocab_size, params.emb_dim)
        if pte is None:
            nn.init.xavier_uniform_(self.word_embeddings.weight)
        else:
            self.word_embeddings.weight.data.copy_(torch.from_numpy(pte))
        # KB Field

        # with open(self.params.entity_tran, 'rb') as f:
        #     transE_embedding = pkl.load(f)
        # self.enti_tran = nn.Embedding.from_pretrained(torch.from_numpy(transE_embedding))

        self.model = HGAT(params)
        self.pooling = Pooling(params)
        self.classifier_sen = nn.Linear(params.node_emb_dim, params.ntags)
        self.classifier_ent = nn.Linear(params.node_emb_dim, params.ntags)

        self.dropout = nn.Dropout(params.dropout, )

        # entity_num = transE_embedding.shape[0]
        # self.gating = GatingMechanism(params) # 这个要放在最后面，尽量少影响随机初始化

    # def forward(self, x_list, adj_list, sentPerDoc, entPerDoc=None):
    def forward(self, documents, ent_desc, doc_lens, ent_lens, adj_lists, feature_lists, sentPerDoc, entiPerDoc=None):
        x_list = []
        embeds_docu = self.word_embeddings(documents)   # sents * max_seq_len * emb
        d = self.text_encoder(embeds_docu, doc_lens)    # sents * max_seq_len * hidden
        d = self.dropout(F.relu_(d))                     # Relu activation and dropout
        x_list.append(d)
        if self.params.node_type == 3 or self.params.node_type == 2:
            embeds_enti = self.word_embeddings(ent_desc)    # sents * max_seq_len * emb
            e = self.enti_encoder(embeds_enti, ent_lens, feature_lists[1])    # sents * max_seq_len * hidden
            e = self.dropout(F.relu_(e))                     # Relu activation and dropout
            x_list.append(e)
        if self.params.node_type == 3 or self.params.node_type == 1:
            t = self.topi_encoder(feature_lists[-1])         # tops * hidden
            x_list.append(t)

        X = self.model(x_list, adj_lists)

        X_s = self.pooling(X[0], sentPerDoc)   # 选择句子的部分
        output = self.classifier_sen(X_s)

        if entiPerDoc is not None:
            # E_trans = self.enti_tran(feature_lists[1])
            E_GCN = X[1]
            # E_KB = self.gating(x_list[1], feature_lists[1])
            E_KB = x_list[1]
            X_e = self.match_encoder(E_GCN, E_KB)  # 选择实体的部分
            X_e = self.pooling(X_e, entiPerDoc)
            X_e = self.classifier_ent(X_e)
            output += X_e
        output = F.softmax(output, dim=1)       # 单分类
        # output = torch.sigmoid(output)        # 多分类
        return output


the submodule (graph neural network)'s class

In [ ]:

class HGAT(nn.Module):
    def __init__(self, params):
        super(HGAT, self).__init__()
        self.para_init()
        self.attention = True
        self.lower_attention = True

        # self.nonlinear = nn.LeakyReLU(0.2)
        self.nonlinear = F.relu_
        nfeat_list = [params.hidden_dim] * ({0: 1, 1: 2, 2: 2, 3: 3}[params.node_type])
        self.ntype = len(nfeat_list)
        nhid = params.node_emb_dim

        self.gc2: nn.ModuleList = nn.ModuleList()
        if not self.lower_attention:
            self.gc1: nn.ModuleList = nn.ModuleList()
            for t in range(self.ntype):
                self.gc1.append( GraphConvolution(nfeat_list[t], nhid, bias=False) )
                self.bias1 = Parameter( torch.FloatTensor(nhid) )
                stdv = 1. / math.sqrt(nhid)
                self.bias1.data.uniform_(-stdv, stdv)
        else:
            self.gc1 = GraphAttentionConvolution(nfeat_list, nhid, gamma=0.1)
        if self.attention:
            self.at1: nn.ModuleList = nn.ModuleList()
            for t in range(self.ntype):
                self.at1.append( SelfAttention(nhid, t, nhid // 2) )
        self.dropout = nn.Dropout(params.dropout)

    def para_init(self):
        self.attention = False
        self.lower_attention = False

    def forward(self, x_list, adj_list, adj_all = None):
        x0 = x_list

        if not self.lower_attention:
            x1 = [None for _ in range(self.ntype)]
            
            for t1 in range(self.ntype):
                x_t1 = []
                for t2 in range(self.ntype):
                    idx = t2
                    x_t1.append( self.gc1[idx](x0[t2], adj_list[t1][t2]) + self.bias1 )
                if self.attention:
                    x_t1, weights = self.at1[t1]( torch.stack(x_t1, dim=1) )
                else:
                    x_t1 = reduce(torch.add, x_t1)
                x_t1 = self.dropout(self.nonlinear(x_t1))
                x1[t1] = x_t1
        else:
            x1 = [None for _ in range(self.ntype)]
            x1_in = self.gc1(x0, adj_list)
            for t1 in range(len(x1_in)):
                x_t1 = x1_in[t1]
                if self.attention:
                    x_t1, weights = self.at1[t1]( torch.stack(x_t1, dim=1) )
                else:
                    x_t1 = reduce(torch.add, x_t1)
                x_t1 = self.dropout(self.nonlinear(x_t1))
                x1[t1] = x_t1

        return x1


    def inference(self, x_list, adj_list, adj_all = None):
        return self.forward(x_list, adj_list, adj_all)



encoder, decoder and pooling function, as shown in the model framework picture in report

In [ ]:

class TextEncoder(Module):
    def __init__(self, params):
        super(TextEncoder, self).__init__()

        self.lstm = LstmEncoder(params.hidden_dim, params.emb_dim)

    def forward(self, embeds, seq_lens):
        return self.lstm(embeds, seq_lens)

class EntityEncoder(Module):
    def __init__(self, params):
        super(EntityEncoder, self).__init__()
        self.lstm = LstmEncoder(params.hidden_dim, params.emb_dim)
        self.gating = GatingMechanism(params)

    def forward(self, embeds, seq_lens, Y):
        X = self.lstm(embeds, seq_lens)
        return self.gating(X, Y)

class Pooling(nn.Module):
    def __init__(self, params):
        super(Pooling, self).__init__()
        self.mode = params.pooling
        self.params = params
        if self.mode == 'max':
            self.pooling = torch.max
        elif self.mode == 'sum':
            self.pooling = torch.sum
        elif self.mode == 'mean':
            self.pooling = torch.mean
        elif self.mode == 'att':
            self.pooling = AttentionPooling(self.params)
        else:
            raise Exception("Unknown pooling mode: {}. (Supported: max, sum, mean, att)".format(self.mode))

    def forward(self, X, sentPerDoc):
        '''
        :param X:           A tensor with shape:  (D1 + D2 + ... + Dn) * H
        :param sentPerDoc:  A tensor with values: [D1, D2, ..., Dn]
        :return:            A tensor with shape:  n * H
        '''
        # weight = [torch.ones((1, i.item()), device=sentPerDoc.device) for i in sentPerDoc]
        # weight = block_diag([m.to_sparse() for m in weight]).to_dense()
        sentPerDoc = sentPerDoc.cpu().numpy().tolist()
        sents = [X[sum(sentPerDoc[: i]): sum(sentPerDoc[: i+1])] for i in range(len(sentPerDoc))]
        output = []
        for s in sents:
            if s.shape[0] == 0:
                output.append(torch.zeros((1, s.shape[1]), device=s.device, dtype=X.dtype))
            else:
                cache = self.pooling(s, dim=0, keepdim=True)
                output.append(cache[0] if isinstance(cache, tuple) else cache)
        output = torch.cat(output, dim=0)
        return output
